# Pattern Driven Stock Analysis

Finding Similar Stocks using Graph Analytis in Neo4j.


# Steps

ETL Script is at Stock Analysis ETL.ipynb

1. [X] Import Stock Data into Neo4j
2. [X] Create [:NEXT] relationship property between nodes
   
4. [ ] Find KNN similarity using trade volume among nodes: Create [:SIMILAR_KNN_VOLUME]  relationship among PRICE Nodes
5. [ ] Community detection based on Louvain algorithm using [:SIMILAR_KNN_VOLUME] relationship: Create VolumeCommunity property
6. [ ] Hierarchical Structures Communities using Label Propagation Algorithm

![Next.png](Next.png)

    MATCH (s:Stock)-[r:TRADING_DAY]->(td:StockTradingDay) 
    WITH s, td
    ORDER BY td.Date ASC
    WITH s, collect(td) as nodes
    CALL graph_util.chain_nodes(nodes, "NEXT_DAY")
    YIELD connections
    RETURN nodes, connections;

  ##  
    MATCH (s:Stock)-[r:TRADING_DAY]->(td:StockTradingDay) 
    WITH s, td
    ORDER BY td.Date ASC
    WITH s, collect(td.Close) as closes
    SET s.close_array = closes


##   Creating Date Nodes

MATCH (td:StockTradingDay)  WITH DISTINCT td.Date as date ORDER BY date ASC WITH date MERGE (d:Date) set d.id = date;


## Relating Date Nodes with Stock Trading day

    MATCH (td:StockTradingDay)  
    MATCH (d:Date) where td.Date = d.id
    MERGE (d)-[:DATE]->(td);

 # Creating Next Date
    
    MATCH (d:Date)
    WITH d
    ORDER BY d.id ASC
    WITH collect(d) as nodes
    CALL graph_util.chain_nodes(nodes, "DATE_NEXT")
    YIELD connections
    RETURN nodes, connections;

    MATCH (d:Date) 
    MATCH (s:Stock) 
    CALL {
        WITH s,d
        OPTIONAL MATCH (s)-[:TRADING_DAY]->(td:StockTradingDay)<-[:DATE]-(d)
        RETURN
        CASE 
            WHEN td.Volume is null THEN 0
            ELSE td.Volume
        END as volume
        }
    WITH s, collect(volume) as volumes
    SET s.volumes_array = volumes
    RETURN s, size(s.volumes_array)

    MATCH (td1:StockTradingDay)-[r:NEXT_DAY]->(td2:StockTradingDay) 
    SET r.volumeGap = (td2.Volume-td1.Volume);

### KNN
Find KNN similarity using trade volume among nodes, using [:SIMILAR_KNN_VOLUME] relationship among PRICE Nodes

In [ ]:
query = """

CALL gds.nodeSimilarity.write.estimate('northwindGraph1', {
  writeRelationshipType: 'SIMILAR',
  writeProperty: 'score'
})
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory

"""

In [ ]:
query = """
CALL gds.nodeSimilarity.stream('northwindGraph1')
YIELD node1, node2, similarity
RETURN gds.util.asNode(node1).customerID AS customer1, gds.util.asNode(node2).customerID AS customer2, similarity
ORDER BY similarity DESCENDING, customer1, customer2
"""

cypher_run(query)

### Community detection
Using Louvain algorithm on [:SIMILAR_KNN_VOLUME] relationship to Create VolumeCommunity property

In [3]:
# https://neo4j.com/docs/graph-data-science/current/algorithms/louvain/

### Label Propagation Algorithm
Hierarchical Structures Communities using

In [2]:
# https://neo4j.com/docs/graph-data-science/current/algorithms/label-propagation/